<a href="https://colab.research.google.com/github/RealMyeong/AIFFELTHON/blob/main/JinMyeong/%EC%9D%BC%EB%B0%98%EB%8C%80%ED%99%94_%EC%B6%94%EA%B0%80_%ED%8C%8C%EC%9D%B4%ED%94%84%EB%9D%BC%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 진행 순서

1. 데이터 불러오기
  - 일반 대화 데이터
  - 공격 대화 데이터
2. 키워드 추출
  - 공격 대화 데이터 키워드 추출
  - 일반 대화 데이터 키워드 추출
    - 화자 구분 x
    - 상담사 제외 고객만 키워드 추출
3. 키워드 비율에 따라 일반대화 추가

### 1. 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# 일반 대화 데이터
normal_conv = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/norm_data_normalization.csv')

# 공격 대화 데이터
attack_conv = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_normalization.csv')

In [3]:
attack_conv

,class,label_sentence,non_label_sentence,binary_class
0,직장 내 괴롭힘 대화,1:길동씨 이번에 이것좀 처리해요\n2:이거 제가 한게 아닌데요\n1:팀에서 내가 ...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...,공격 대화
1,기타 괴롭힘 대화,1:야 너 앞니 왜 그렇게 튀어나왔냐\n2:태어날때부터 그랬어 물어보지마\n1:아 ...,야 너 앞니 왜 그렇게 튀어나왔냐\n태어날 때부터 그랬어 물어보지 마\n아 그럼 태...,공격 대화
2,갈취 대화,1:원후야 학원 교재 사야되는데 8만원만 주면 안돼\n2:미안 나 오늘은 진짜 돈이...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n미안 나 오늘은 진짜 돈이 ...,공격 대화
3,갈취 대화,1:너 저번에 술먹은 날 기억해\n2:아니 왜\n1:야 300만원만 가져와\n2:뭔...,너 저번에 술 먹은 날 기억해\n아니 왜\n야 300만 원만 가져와\n뭔 소리야 ...,공격 대화
4,협박 대화,1:너 또 내말 무시하냐\n1:이새끼 널 좆으로 보나본데\n2:아냐 진짜 시간이 없...,너 또 내말 무시하냐\n이 새끼 널 좆으로 보나 본데\n아냐 진짜 시간이 없었어 미...,공격 대화
...,...,...,...,...
3865,기타 괴롭힘 대화,1:준하야 넌 대가리가 왜이렇게 크냐\n2:내 머리가 뭐\n1:밥먹으면 대가리만 크...,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,공격 대화
3866,갈취 대화,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,공격 대화
3867,직장 내 괴롭힘 대화,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,공격 대화
3868,갈취 대화,1:누구맘대로 여기서 장사하래\n2:이게 무슨일입니까\n1:남의 구역에서 장사하려면...,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,공격 대화


In [4]:
# attack_conv.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_normalization.csv', index=False)

In [5]:
normal_conv

,type,non_label_sentence,label_sentence,class,binary_class
0,고객센터,네 침구 추가를 해야 할 것 같아서요\n네 1세트가 필요하신 건가요\n네 맞습니다 ...,1:반갑습니다 입니다\n2:네 침구 추가를 해야 할 것 같아서요\n1:네 1세트가 ...,일반 대화,일반 대화
1,고객센터,네 예약하려면 어떻게 해야 하나요\n네 회원 가입 후 로그인 하셔서 예약하시면 되세...,1:반갑습니다 입니다\n2:네 예약하려면 어떻게 해야 하나요\n1:네 회원 가입 후...,일반 대화,일반 대화
2,고객센터,네 객실 인원이 추가될 경우엔 어떻게 하면 되나요\n아 일반적인 스탠다드 디럭스 객...,1:반갑습니다 입니다\n2:네 객실 인원이 추가될 경우엔 어떻게 하면 되나요\n1:...,일반 대화,일반 대화
3,고객센터,네 듣고 보니 코인 세탁 방이 있다고 하던 데요\n아 코인 런더리룸이요\n네 어디에...,1:반갑습니다 입니다\n2:네 듣고 보니 코인 세탁 방이 있다고 하던 데요\n1:아...,일반 대화,일반 대화
4,고객센터,네 혹시 부대 시설로 편의점이 있나요\n네 호텔 1층에 위치해 있습니다\n아 어떤 ...,1:반갑습니다 입니다\n2:네 혹시 부대 시설로 편의점이 있나요\n1:네 호텔 1층...,일반 대화,일반 대화
...,...,...,...,...,...
108626,카카오,오늘 친구네 강아지 산책하는데 따라갔다 왔다\n와 부러워 전에 그 바닷가 사는 애\...,1:오늘 친구네 강아지 산책하는데 따라갔다 왔다\n2:와 부러워 전에 그 바닷가 사...,일반 대화,일반 대화
108627,카카오,요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n갑자기 집이랑 만 왔다 갔다...,1:요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n2:갑자기 집이랑 만 왔...,일반 대화,일반 대화
108628,카카오,주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n뭐 바퀴 달린 거 그...,1:주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n2:뭐 바퀴 달린...,일반 대화,일반 대화
108629,카카오,우리 집 뒤에 중국집 가지마\n왜 너 거기 좋아했잖아 맛있다며\n다 다음 달까지 거...,1:우리 집 뒤에 중국집 가지마\n2:왜 너 거기 좋아했잖아 맛있다며\n1:다 다음...,일반 대화,일반 대화


In [6]:
import numpy as np
idx = []
batch_idx = []

for i in range(len(normal_conv)):
  batch_idx.append(i)
  if len(batch_idx)==10:
    idx.append(batch_idx)
    batch_idx = []

In [7]:
len(idx)

10863

In [8]:
batch_sen = []
for i in idx:
  a = normal_conv.iloc[i].reset_index(drop=True).non_label_sentence
  sen = []
  for j in a:
    t = j.split('\n')
    sen += t

  batch_sen.append(sen)

In [9]:
batch_sen[0]

['네 침구 추가를 해야 할 것 같아서요',
 '네 1세트가 필요하신 건가요',
 '네 맞습니다 추가 요금은 얼마인가요',
 '네 1만 원이시고요 투숙기간 과는 무관합니다',
 '네 알겠습니다 침구문의는 어디다 말씀드려야 할까요',
 '네 프론트에다가 사전에 말씀 부탁 드릴게요',
 '네 알겠습니다',
 '네  오염 시에는 세탁료 5천 원이 발생하실 수 있으세요',
 '네 알겠습니다',
 '네 예약하려면 어떻게 해야 하나요',
 '네 회원 가입 후 로그인 하셔서 예약하시면 되세요',
 '네 알겠습니다 객실엔 세면 도구가 있나요',
 '네 타월과 욕실 용품은 있지만 개인 용품은 챙겨오셔야 합니다',
 '아 면도기와 칫솔 치약은 준비해야겠네요',
 '네 더 궁금하신 부분이 있으실까요',
 '객실에서는 취사가 불가 할까요',
 '네 휴대용버너 가스렌지 등 반입이 불가능합니다',
 '네 알겠습니다',
 '네 객실 인원이 추가될 경우엔 어떻게 하면 되나요',
 '아 일반적인 스탠다드 디럭스 객실은 인원 초과로 입실이 불가하십니다',
 '아 전 로열스위트 객실인데 이 객실도 안되나요',
 '아 로열스위트는 인원 추가와 침구 추가 시 비용이 발생됩니다',
 '아 인원 추가 시에는 얼마인가요',
 '네 1박당 1인은 1만원이세요',
 '그럼 침구 추가 시에는 어떻게 되나요',
 '네 1채 당 1만원',
 '네 응대해주셔서',
 '네 듣고 보니 코인 세탁 방이 있다고 하던 데요',
 '아 코인 런더리룸이요',
 '네 어디에 위치해 있나요',
 '네 지하1층에 있습니다',
 '아 운영 시간은 언제까지 하나요',
 '아 24시간으로 운영되고 있습니다',
 '네 알려주셔서',
 '네 혹시 더 궁금하신 사항 있으시면 전화 주세요',
 '네 응대해주셔서',
 '네 혹시 부대 시설로 편의점이 있나요',
 '네 호텔 1층에 위치해 있습니다',
 '아 어떤 품목을 판매하나요',
 '네 담배나 라면 과자 등 판매하고 있습니다',
 '아 운영 시간은 언제까지 인가요',
 '네 지금은 성수기라 

In [10]:
len(batch_sen)

10863

### 2. 키워드 추출

1. 공격 대화 키워드 추출


In [11]:
!pip install krwordrank

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
def keyword(class_name, data, n=20):
    genre = data[data['class']==class_name].reset_index(drop=True).label_sentence

    attack, damaged = [], []

    for e in genre:
        e = e.split('\n')
        for e_ in e:
            if e_[0] == '1':
                attack.append(e_[2:])
            else:
                damaged.append(e_[2:])

    from krwordrank.sentence import summarize_with_sentences

    texts = attack
    keywords_att, sents = summarize_with_sentences(texts, num_keywords=100, 
                                           num_keysents=10, max_length=10, diversity=0.5)

    texts = damaged
    keywords_dam, sents = summarize_with_sentences(texts, num_keywords=100, 
                                           num_keysents=10, max_length=10, diversity=0.5)

    print('{}의 가해자 :'.format(class_name),keywords_att.keys())
    print()
    print('{}의 피해자 :'.format(class_name),keywords_dam.keys())
    
    keywords_att = list(keywords_att.keys())
    keywords_dam = list(keywords_dam.keys())
    
    target_attack, target_damaged = [], []

    for i in range(100):
        if (keywords_att[i] not in keywords_dam) : target_attack.append(keywords_att[i])
        if len(target_attack) == n: break 

    for i in range(100):
        if (keywords_dam[i] not in keywords_att) : target_damaged.append(keywords_dam[i])
        if len(target_damaged) == n: break
    
    print()
    print('{}의 가해자(중복제거) :'.format(class_name),target_attack)
    print()
    print('{}의 피해자(중복제거) :'.format(class_name),target_damaged)

    return [target_attack, target_damaged]

In [13]:
extort_att, extort_vic = keyword('갈취 대화', attack_conv)

갈취 대화의 가해자 : dict_keys(['내가', '그럼', '아니', '그래', '지금', '빌려', '그냥', '내놔', '이거', '빨리', '진짜', '돈이', '우리', '친구', '오늘', '없어', '나도', '니가', '나오면', '제가', '여기', '있는', '그러', '보내', '너무', '가져', '당장', '좋은', '있어', '10', '어떻게', '이번', '거기', '그거', '그렇게', '사람', '알아', '나한테', '뒤져서', '얼마', '어이', '내일', '한번', '엄마', '당신', '이렇게', '가지고', '근데', '싫으면', '줄래', '무슨', '그건', '누가', '얼른', '말이', '맞고', '너가', '이게', '돈을', '돈좀', '많이', '전에', '지갑', '그런', '바로', '어디', '말고', '생각', '주면', '하나', '줘봐', '줄게', '가방', '새끼', '너네', '뭐야', '다른', '입금', '저기', '잠깐', '좋아', '내놓', '돌려', '하면', '다시', '그게', '신고', '없는', '거짓말', '학교', '하는', '조용히', '이런', '이제', '저희', '이건', '쓰고', '필요', '이리', '안돼'])

갈취 대화의 피해자 : dict_keys(['제발', '아니', '진짜', '안돼', '없어', '이거', '무슨', '내가', '돈이', '정말', '미안', '제가', '그래', '지금', '그건', '어떻게', '알겠', '여기', '나도', '그러', '엄마', '죄송', '그게', '빌려', '그럼', '돌려', '이게', '이건', '이번', '저희', '그거', '그런', '없는', '저번에', '한번만', '우리', '이러', '오늘', '너무', '안되', '이제', '빨리', '갑자기', '그냥', '근데', '있어', '얼마', '가져', '돈을', '알았어', '누구', '드릴게요', '당장', '조금만', '주세요', '

In [14]:
work_att, work_vic = keyword('직장 내 괴롭힘 대화', attack_conv)

직장 내 괴롭힘 대화의 가해자 : dict_keys(['내가', '아니', '그래', '지금', '오늘', '그럼', '자네', '그렇게', '우리', '그냥', '김대리', '무슨', '이거', '이렇게', '사람', '하는', '진짜', '일을', '이번', '하고', '어떻게', '요즘', '너무', '없어', '내일', '그런', '그러', '제대로', '말이', '이런', '알아', '다른', '일이', '휴가', '있어', '빨리', '보고', '아주', '죄송', '니가', '하나', '생각', '다시', '누가', '못하', '어디', '같이', '하면', '여기', '그게', '뭐가', '해야지', '이게', '가서', '하지', '근데', '업무', '똑바로', '너가', '상사', '생활', '싶어', '커피', '말고', '거야', '어제', '들어', '앞으로', '못해', '나한테', '뭐야', '없는', '있는', '일도', '아직', '부탁', '한번', '얼마나', '승진', '이제', '너는', '이래서', '그건', '제가', '혼자', '맨날', '주말에', '회식', '나랑', '같은', '나도', '있나', '부장님', '집에', '하라', '안돼', '시간', '나가', '퇴근', '이대리'])

직장 내 괴롭힘 대화의 피해자 : dict_keys(['제가', '죄송', '아니', '부장님', '아닙니다', '무슨', '너무', '과장님', '정말', '다시', '지금', '말씀', '저도', '하겠습니다', '그건', '그래도', '일이', '팀장님', '오늘', '그게', '저는', '이건', '알겠', '그런', '같습니다', '있어서', '대리님', '열심히', '빨리', '사장님', '업무', '아닌', '내일', '그럼', '진짜', '없습니다', '이번', '그렇게', '다른', '있습니다', '갑자기', '하고', '그러', '앞으로', '어떻게', '휴가', '아직', '아뇨', '바로', '혹시', '

In [15]:
threat_att, threat_vic = keyword('협박 대화', attack_conv)

협박 대화의 가해자 : dict_keys(['내가', '니가', '지금', '진짜', '그래', '아니', '그냥', '그럼', '죽여', '우리', '너가', '있어', '당장', '죽어', '사람', '어떻게', '하나', '알아', '당신', '빨리', '그러', '나도', '너도', '그렇게', '없어', '죽고', '어디', '여기', '이렇게', '전에', '이게', '나한테', '오늘', '같이', '이제', '무슨', '가족', '하면', '자꾸', '정신', '아주', '거야', '생각', '있는', '한번', '말이', '칼로', '않으면', '이거', '새끼', '네가', '나랑', '헤어', '죽이', '너네', '정말', '너는', '죽는', '하지', '다시', '그런', '뭐가', '보고', '싶어', '아직', '하는', '장기', '들어', '얼마나', '신고', '나는', '잘못', '죽을', '알지', '너희', '가만', '이미', '몰라', '했지', '계속', '죽일', '이런', '집에', '조용히', '바로', '모르', '너무', '하고', '말을', '소리', '근데', '좋은', '목숨', '그게', '너를', '싫으면', '니네', '다른', '맞아', '아들'])

협박 대화의 피해자 : dict_keys(['제발', '아니', '내가', '정말', '제가', '무슨', '지금', '우리', '진짜', '죄송', '살려', '그래', '그러', '어떻게', '잘못', '미안', '그만', '너무', '그렇게', '그게', '경찰에', '그냥', '이러', '그런', '이제', '그건', '사람', '알겠', '신고', '이거', '시간을', '건들지', '주세요', '없어', '당신', '한번만', '아닙니다', '협박', '나도', '나는', '안돼', '조금만', '저는', '선생님', '다시', '그럼', '니가', '너가', '당장', '여기', '하지', '자꾸', '진정', '어디', '마세요', 

In [16]:
other_att, other_vic = keyword('기타 괴롭힘 대화', attack_conv)

기타 괴롭힘 대화의 가해자 : dict_keys(['아니', '내가', '진짜', '그냥', '그래', '니가', '우리', '그럼', '여기', '지금', '너무', '그렇게', '무슨', '너가', '이거', '어떻게', '빨리', '그러', '사람', '얼굴', '어디', '뭐가', '이렇게', '너는', '냄새', '친구', '그런', '장애', '새끼', '돼지', '엄마', '이게', '없어', '오늘', '알아', '못생', '나는', '하는', '하고', '너도', '애들', '기분', '있어', '하지', '뭐야', '너네', '같이', '저기', '다니', '그거', '근데', '다른', '말이', '다시', '존나', '누가', '환불', '남자', '너랑', '그만', '싫어', '따라', '이런', '부모', '집에', '찐따', '제가', '당장', '하나', '거지', '나도', '에이', '학교', '생각', '말고', '것도', '하면', '이제', '몰라', '다리', '없는', '못하', '그게', '완전', '같은', '너한테', '그리고', '장난', '사장', '얼마나', '있는', '맨날', '공부', '병신', '좋아', '소리', '보고', '아가씨', '말하', '정말'])

기타 괴롭힘 대화의 피해자 : dict_keys(['아니', '내가', '손님', '무슨', '그만', '제발', '제가', '진짜', '죄송', '지금', '그런', '너무', '그래', '우리', '하지', '미안', '그렇게', '그냥', '그러', '그게', '말이', '나도', '나한테', '정말', '저희', '어떻게', '여기', '그건', '있어', '이거', '자꾸', '사람', '그럼', '너네', '왜그', '하는', '선생님', '기분', '오늘', '엄마', '혹시', '잘못', '다시', '다른', '나는', '말씀', '니가', '이렇게', '싫어', '이러', '친구', '알겠', '이제', '말을', '없어', 

위 키워드를 바탕으로 중복을 제거한 가해자들의 키워드를 따로 저장해줌.

피해자는 대부분 `네? 죄송합니다. 미안 ` 등의 키워드가 나와서 비슷한 모습을 볼 수 있음.

공격적인 내용의 대부분은 가해자가 하는 말에의해 결정된다고 볼 수 있음.

In [17]:
att_keyword = extort_att + work_att + threat_att + other_att

In [18]:
# 중복 제거
att_keyword = list(set(att_keyword))
len(att_keyword)

65

In [19]:
batch_sen[1]

['네 객실 예약 변경이나 취소는 어떻게 하나요',
 '네 홈페이지 우측에 예약 확인 취소 버튼을 눌러주시면 되십니다',
 '아 예약을 변경하는 경우엔 확인 수정을 누르면 되나요',
 '네 화면 하단에 예약하기에서 변경이 가능하세요',
 '그러면 취소하고 싶을 땐 어떻게 해야 하나요',
 '네 취소 버튼을 누르시면 취소 완료가 되십니다',
 '네 간단하고 편리하네요',
 '네 더 궁금하신 사항이 있으실까요',
 '아니요 해결 되었네요',
 '전화 주셔서',
 '네 객실 체크인은 몇 시부터 가능하나요',
 '네 비수기에는 오후 2시 성수기나 연휴엔 오후 3시에 가능하세요',
 '아 제가 늦게 도착할 것 같은데 밤 10시에 입실 하게 되면 어쩌죠',
 '아 그럴 경우엔 사전에 미리 연락 주시기 바랄게요',
 '네 그럼요 퇴실 시간은 어떻게 되나요',
 '네 비수기엔 오후 12시 성수기나 연휴엔 오전 11시 까지세요',
 '아 그렇군요 혹시 퇴실 시간 초과 시엔 추가 비용이 얼마인가요',
 '네 오후 3시까지 연장 가능하시고 시간 당 1만원이십니다',
 '네 알겠습니다',
 '전화 주셔서',
 '네 숙박할 때요 정원 초과에 대한 추가 요금은 어떻게 되나요',
 '네 만13세 이상은 1만원이시고요',
 '아 만 7세에서 12세 이하는 요',
 '네 5천 원 발생 되시고요 미취학 아동은 무료',
 '아 그럼요 정원 몇 명까지 추가가 가능하나요',
 '네 정원 2명까지만 가능하세요',
 '그럼 그 금액에 비품도 포함되나요',
 '아니요 초과 인원에 대한 별도의 비품은 제공되지 않습니다',
 '네 알겠습니다',
 '전화 주셔서',
 '네 예약은 몇 일 전부터 예약 가능하나요',
 '네 이용일 전달 1일부터 예약 접수가 가능하십니다',
 '아 주말과 국경일엔 안되는 건가요',
 '네 그때는 예약 접수를 받고 있지 않습니다',
 '네 알겠습니다 그럼 성수기 때는 어떻게 되나요',
 '네 여름 겨울 기간은 홈페이지나 문자로 공지 드리고 있어요',
 '네 알겠습니다',
 '더 궁

In [20]:
def select_normal_conversation(sen, att_keyword, threshold=0.6):

  
  from krwordrank.sentence import summarize_with_sentences

  texts = sen
  keywords_client, sents = summarize_with_sentences(texts, num_keywords=20, 
                                           num_keysents=5, max_length=10, diversity=0.3)

  keywords_client = list(keywords_client.keys())
  keywords_att = att_keyword

  length = len(keywords_client)

  clean_keywords = []
  for word in keywords_client:
    if not word in keywords_att:
      clean_keywords.append(word)
  
  clean_ratio = len(clean_keywords) / length
  if clean_ratio > threshold:
    return True
  else:
    return False

In [21]:
select_batch_idx = []

for i in range(len(batch_sen)):
  try:
    if select_normal_conversation(batch_sen[i], att_keyword):
      select_batch_idx.append(i)
  except:
    continue

In [22]:
len(select_batch_idx)

10581

In [24]:
normal_idx = []
for i in select_batch_idx:
  normal_idx += idx[i]

In [26]:
len(normal_idx)

105810

In [27]:
filtered_normal_conv = normal_conv.iloc[normal_idx]
filtered_normal_conv

,type,non_label_sentence,label_sentence,class,binary_class
0,고객센터,네 침구 추가를 해야 할 것 같아서요\n네 1세트가 필요하신 건가요\n네 맞습니다 ...,1:반갑습니다 입니다\n2:네 침구 추가를 해야 할 것 같아서요\n1:네 1세트가 ...,일반 대화,일반 대화
1,고객센터,네 예약하려면 어떻게 해야 하나요\n네 회원 가입 후 로그인 하셔서 예약하시면 되세...,1:반갑습니다 입니다\n2:네 예약하려면 어떻게 해야 하나요\n1:네 회원 가입 후...,일반 대화,일반 대화
2,고객센터,네 객실 인원이 추가될 경우엔 어떻게 하면 되나요\n아 일반적인 스탠다드 디럭스 객...,1:반갑습니다 입니다\n2:네 객실 인원이 추가될 경우엔 어떻게 하면 되나요\n1:...,일반 대화,일반 대화
3,고객센터,네 듣고 보니 코인 세탁 방이 있다고 하던 데요\n아 코인 런더리룸이요\n네 어디에...,1:반갑습니다 입니다\n2:네 듣고 보니 코인 세탁 방이 있다고 하던 데요\n1:아...,일반 대화,일반 대화
4,고객센터,네 혹시 부대 시설로 편의점이 있나요\n네 호텔 1층에 위치해 있습니다\n아 어떤 ...,1:반갑습니다 입니다\n2:네 혹시 부대 시설로 편의점이 있나요\n1:네 호텔 1층...,일반 대화,일반 대화
...,...,...,...,...,...
108625,카카오,일본 진짜 가고 싶다\n나 요새 맨날 이 애기 하잖아\n인정 솔직히 너 일본에서 명...,1:일본 진짜 가고 싶다\n1:나 요새 맨날 이 애기 하잖아\n2:인정 솔직히 너 ...,일반 대화,일반 대화
108626,카카오,오늘 친구네 강아지 산책하는데 따라갔다 왔다\n와 부러워 전에 그 바닷가 사는 애\...,1:오늘 친구네 강아지 산책하는데 따라갔다 왔다\n2:와 부러워 전에 그 바닷가 사...,일반 대화,일반 대화
108627,카카오,요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n갑자기 집이랑 만 왔다 갔다...,1:요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n2:갑자기 집이랑 만 왔...,일반 대화,일반 대화
108628,카카오,주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n뭐 바퀴 달린 거 그...,1:주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n2:뭐 바퀴 달린...,일반 대화,일반 대화


In [28]:
cs_data = filtered_normal_conv[filtered_normal_conv['type']=='고객센터']
cs_data

,type,non_label_sentence,label_sentence,class,binary_class
0,고객센터,네 침구 추가를 해야 할 것 같아서요\n네 1세트가 필요하신 건가요\n네 맞습니다 ...,1:반갑습니다 입니다\n2:네 침구 추가를 해야 할 것 같아서요\n1:네 1세트가 ...,일반 대화,일반 대화
1,고객센터,네 예약하려면 어떻게 해야 하나요\n네 회원 가입 후 로그인 하셔서 예약하시면 되세...,1:반갑습니다 입니다\n2:네 예약하려면 어떻게 해야 하나요\n1:네 회원 가입 후...,일반 대화,일반 대화
2,고객센터,네 객실 인원이 추가될 경우엔 어떻게 하면 되나요\n아 일반적인 스탠다드 디럭스 객...,1:반갑습니다 입니다\n2:네 객실 인원이 추가될 경우엔 어떻게 하면 되나요\n1:...,일반 대화,일반 대화
3,고객센터,네 듣고 보니 코인 세탁 방이 있다고 하던 데요\n아 코인 런더리룸이요\n네 어디에...,1:반갑습니다 입니다\n2:네 듣고 보니 코인 세탁 방이 있다고 하던 데요\n1:아...,일반 대화,일반 대화
4,고객센터,네 혹시 부대 시설로 편의점이 있나요\n네 호텔 1층에 위치해 있습니다\n아 어떤 ...,1:반갑습니다 입니다\n2:네 혹시 부대 시설로 편의점이 있나요\n1:네 호텔 1층...,일반 대화,일반 대화
...,...,...,...,...,...
91299,고객센터,네 궁금한 게 있어서요\n네 회원님 무엇이 궁금하신가요\n에서 파일 삭제는 어떻게...,1: 상담원 입니다 무엇을 도와드릴까요\n2:네 안녕하세요 궁금한 게 있어서요\n1...,일반 대화,일반 대화
91300,고객센터,제가 궁금한게 있어서 전화 드렸어요\n네 말씀해보세요 답변해 드리겠습니다\n다운로드...,1:반갑습니다 상담원 입니다 무엇을 도와드릴까요\n2:안녕하세요 제가 궁금한게 있어...,일반 대화,일반 대화
91301,고객센터,등급 전체를 다운 받고 싶은데 안되네요\n모든 세트를 전부 다운 받을 수는 없어요\...,1:안녕하세요 상담원 입니다 무엇을 도와드릴까요\n2:안녕하세요 등급 전체를 다운 ...,일반 대화,일반 대화
91302,고객센터,제가 궁금한 게 있어서 전화드렸어요\n네 말씀해 보세요 답변해 드리겠습니다\n음원 ...,1:안녕하세요 상담원 입니다 무엇을 도와드릴까요\n2:안녕하세요 제가 궁금한 게 있...,일반 대화,일반 대화


In [29]:
kakao_data = filtered_normal_conv[filtered_normal_conv['type'] == '카카오']
kakao_data

,type,non_label_sentence,label_sentence,class,binary_class
8349,카카오,중고 거래 해본 적 있어\n응응 당근거래 자주 해\n난 당근마켓 몇 번 해봤어\n당...,1:중고 거래 해본 적 있어\n2:응응 당근거래 자주 해\n1:난 당근마켓 몇 번 ...,일반 대화,일반 대화
8350,카카오,연예인은 다 이미지야\n그렇ㅈ 공인이잖아 완젼\n실망한 연예인 진짜 많아 서예...,1:연예인은 다 이미지야\n2:그렇ㅈ 공인이잖아 완젼\n1:실망한 연예인 진짜 많아...,일반 대화,일반 대화
8351,카카오,영화 보는 거 좋아해\n응응 영화 엄청 좋아하지\n난 영화 보는 거 좋아해 하하\n...,1:영화 보는 거 좋아해\n2:응응 영화 엄청 좋아하지\n1:난 영화 보는 거 좋아...,일반 대화,일반 대화
8352,카카오,급 출출한데 야식 고\n좀 전에 배달 시켰어 난\n뭐 시켰어 메뉴는\n족발 나...,1:급 출출한데 야식 고\n2:좀 전에 배달 시켰어 난\n1:뭐 시켰어 메뉴는\n2...,일반 대화,일반 대화
8353,카카오,우리 아파트도 외벽 색깔 좀\n맞아 외벽 진짜 이상해\n아파트 외관이 넘 후지다\...,1:우리 아파트도 외벽 색깔 좀\n2:맞아 외벽 진짜 이상해\n1:아파트 외관이 넘...,일반 대화,일반 대화
...,...,...,...,...,...
108625,카카오,일본 진짜 가고 싶다\n나 요새 맨날 이 애기 하잖아\n인정 솔직히 너 일본에서 명...,1:일본 진짜 가고 싶다\n1:나 요새 맨날 이 애기 하잖아\n2:인정 솔직히 너 ...,일반 대화,일반 대화
108626,카카오,오늘 친구네 강아지 산책하는데 따라갔다 왔다\n와 부러워 전에 그 바닷가 사는 애\...,1:오늘 친구네 강아지 산책하는데 따라갔다 왔다\n2:와 부러워 전에 그 바닷가 사...,일반 대화,일반 대화
108627,카카오,요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n갑자기 집이랑 만 왔다 갔다...,1:요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n2:갑자기 집이랑 만 왔...,일반 대화,일반 대화
108628,카카오,주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n뭐 바퀴 달린 거 그...,1:주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n2:뭐 바퀴 달린...,일반 대화,일반 대화


In [30]:
filtered_normal_conv.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/real_norm_data', index=False)

### 3. 비율에 따라 일반대화 추가

먼저 카카오톡 대화와 고객센터 대화의 비율은 3:1로 정했습니다.

기존의 공격대화에서 볼 때 직장 내 괴롭힘 클래스에 해당하는 대화를 제외하고는 대부분 가벼운 말투(야, 너, 가져와, 해라 등등)입니다.

추가할 일반 데이터인 카카오톡 데이터와 고객센터 데이터를 비교해보면 카카오톡 데이터가 비교적 가벼운 말투로 이루어져있고, 고객센터 대화가 조금 더 딱딱한 말투로 이루어져 있기 때문에 이에 맞춰서 데이터의 비율을 정했습니다.

먼저 공격대화를 일반 대화와 비슷하게 한글 숫자 제외하고 전부 전처리 해줍니다.

In [31]:
attack_conv

,class,label_sentence,non_label_sentence,binary_class
0,직장 내 괴롭힘 대화,1:길동씨 이번에 이것좀 처리해요\n2:이거 제가 한게 아닌데요\n1:팀에서 내가 ...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...,공격 대화
1,기타 괴롭힘 대화,1:야 너 앞니 왜 그렇게 튀어나왔냐\n2:태어날때부터 그랬어 물어보지마\n1:아 ...,야 너 앞니 왜 그렇게 튀어나왔냐\n태어날 때부터 그랬어 물어보지 마\n아 그럼 태...,공격 대화
2,갈취 대화,1:원후야 학원 교재 사야되는데 8만원만 주면 안돼\n2:미안 나 오늘은 진짜 돈이...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n미안 나 오늘은 진짜 돈이 ...,공격 대화
3,갈취 대화,1:너 저번에 술먹은 날 기억해\n2:아니 왜\n1:야 300만원만 가져와\n2:뭔...,너 저번에 술 먹은 날 기억해\n아니 왜\n야 300만 원만 가져와\n뭔 소리야 ...,공격 대화
4,협박 대화,1:너 또 내말 무시하냐\n1:이새끼 널 좆으로 보나본데\n2:아냐 진짜 시간이 없...,너 또 내말 무시하냐\n이 새끼 널 좆으로 보나 본데\n아냐 진짜 시간이 없었어 미...,공격 대화
...,...,...,...,...
3865,기타 괴롭힘 대화,1:준하야 넌 대가리가 왜이렇게 크냐\n2:내 머리가 뭐\n1:밥먹으면 대가리만 크...,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,공격 대화
3866,갈취 대화,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,공격 대화
3867,직장 내 괴롭힘 대화,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,공격 대화
3868,갈취 대화,1:누구맘대로 여기서 장사하래\n2:이게 무슨일입니까\n1:남의 구역에서 장사하려면...,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,공격 대화


In [ ]:
# def preprocess(sen):
#   sen = sen.split('\n')
  
#   clean_sen = []
#   for i in sen:
#     i = i.strip()
#     i = re.sub(r'[^0-9가-힣:]', ' ', i)
#     i = i.strip()
#     clean_sen.append(i)

#   return '\n'.join(clean_sen)

In [ ]:
# attack_conv['label_sentence'] = attack_conv['label_sentence'].apply(lambda x : preprocess(x))
# attack_conv

In [ ]:
# attack_conv['non_label_sentence'] = attack_conv['non_label_sentence'].apply(lambda x : preprocess(x))
# attack_conv

1. 일반 대화 1000개 추가 (카카오톡 750 : 고객센터 250)

In [32]:
kakao_750 = kakao_data.sample(750)
kakao_750

,type,non_label_sentence,label_sentence,class,binary_class
16800,카카오,역시 일기예보는 맞은 적이 한 번도 없어\n요즘 잘 맞는 거 같던데\n어제 오늘 비...,1:역시 일기예보는 맞은 적이 한 번도 없어\n2:요즘 잘 맞는 거 같던데\n1:어...,일반 대화,일반 대화
14352,카카오,요즘엔 군대도 많이 편해 졌다고 하더라\n맞아\n다들 그 얘기 많이 하더라\n모든 ...,1:요즘엔 군대도 많이 편해 졌다고 하더라\n2:맞아\n2:다들 그 얘기 많이 하더...,일반 대화,일반 대화
103117,카카오,나 피아노 학원이나 다닐까\n갑자기 뭔 놈에 피아노 학원\n안 친 세월이 몇 년이야...,1:나 피아노 학원이나 다닐까\n2:갑자기 뭔 놈에 피아노 학원\n1:안 친 세월이...,일반 대화,일반 대화
51199,카카오,우리 강아지가 자꾸 아무데나 똥을 싸\n교육을 시켜야겠다\n배변 훈련은 어디서 시켜...,1:우리 강아지가 자꾸 아무데나 똥을 싸\n2:교육을 시켜야겠다\n1:배변 훈련은 ...,일반 대화,일반 대화
27060,카카오,언니 혼내는 거 아녀요\n원래 혼내면서 배우는 거지\n난 솔직히 아소비 보내고 싶...,1:언니 혼내는 거 아녀요\n2:원래 혼내면서 배우는 거지\n1:난 솔직히 아소비 ...,일반 대화,일반 대화
...,...,...,...,...,...
48469,카카오,방금 번쩍한거야\n번개 내려쳤나 봐\n갑자기 비 쏟아지네\n분명 일기예보는 화창하다...,1:방금 번쩍한거야\n2:번개 내려쳤나 봐\n1:갑자기 비 쏟아지네\n2:분명 일기...,일반 대화,일반 대화
47785,카카오,예전에 공원에 갔다가 큰 연못 위에 원앙 둘이 떠 있는 걸 봤어\n아 나도 본적 ...,1:예전에 공원에 갔다가 큰 연못 위에 원앙 둘이 떠 있는 걸 봤어\n2:아 나도 ...,일반 대화,일반 대화
58648,카카오,홈 트레이닝 자세 추천해줘\n나는 스쿼트 많이 해\n안마의자 사려는데 안마의자 이용...,1:홈 트레이닝 자세 추천해줘\n2:나는 스쿼트 많이 해\n1:안마의자 사려는데 안...,일반 대화,일반 대화
55098,카카오,아 갑자기 국수먹고싶다\n이시간에 무슨 국수여\n그 예전에 근처에 옥동식당 국수\n...,1:아 갑자기 국수먹고싶다\n2:이시간에 무슨 국수여\n1:그 예전에 근처에 옥동식...,일반 대화,일반 대화


In [33]:
cs_250 = cs_data.sample(250)
cs_250

,type,non_label_sentence,label_sentence,class,binary_class
81422,고객센터,환불하실 제품이 인가요\n네 이 맞습니다\n주문하실 때 두 가지 을 같이 해주셨는데...,1: 환불하실 제품이 인가요\n2:네 이 맞습니다\n1:주문하실 때 두 가지 을 같...,일반 대화,일반 대화
76474,고객센터,무이자 할부가 다른 을 같이 결제하면 어떻게 되나요\n무이자 할부 개월 수가 다른 ...,1:안녕하세요 입니다\n2:무이자 할부가 다른 을 같이 결제하면 어떻게 되나요\n1...,일반 대화,일반 대화
69563,고객센터,철도경찰이 경조사에 찾아온 손님에게요\n가액기준 이하의 경조사비를 받고 나서요\n가...,1:안녕하세요 무엇을 도와드릴까요\n2:철도경찰이 경조사에 찾아온 손님에게요\n2:...,일반 대화,일반 대화
64348,고객센터,네 선생님\n인조잔디 입찰 관계 문의주셨던 선생님 맞으십니까 잠시 통화 가능하십니까...,1:안녕하세요 입니다 네 선생님\n1:인조잔디 입찰 관계 문의주셨던 선생님 맞으십니...,일반 대화,일반 대화
81338,고객센터,배송일 확인은 어디서 하나요\n피시의 경우에는 로그인 후 상단에 마이페이지 구매내역...,1:반갑습니다 입니다\n2:안녕하세요 배송일 확인은 어디서 하나요\n1:피시의 경우...,일반 대화,일반 대화
...,...,...,...,...,...
70454,고객센터,네 건의할 게 있어서요\n네 말씀해주십시오 어떤 문의십니까\n인천 송도 달빛공원에 ...,1:안녕하세요 상담원 입니다\n2:네 건의할 게 있어서요\n1:네 말씀해주십시오 어...,일반 대화,일반 대화
82877,고객센터,온라인 등급에 대해 궁금합니다\n온라인 등급 어떤 내용이세요\n선정 기간을 알려주세...,1:안녕하세요 상담원 입니다\n2:온라인 등급에 대해 궁금합니다\n1:온라인 등급 ...,일반 대화,일반 대화
1880,고객센터,기분 좋은 여행을 위한 여수관광문화 안내 센터\n제가 여수 여행을 준비 중인데요\n...,1:기분 좋은 여행을 위한 여수관광문화 안내 센터 입니다\n2:제가 여수 여행을 준...,일반 대화,일반 대화
86429,고객센터,네 수고가 많으세요 혹시 말하기 학습은 어떻게 하나요\n네 저의 어플을 통해 말하기...,1:안녕하세요 상담원 입니다 무엇을 도와드릴까요\n2:네 수고가 많으세요 혹시 말하...,일반 대화,일반 대화


In [34]:
norm_1000 = pd.concat([kakao_750, cs_250], ignore_index=True)
norm_1000

,type,non_label_sentence,label_sentence,class,binary_class
0,카카오,역시 일기예보는 맞은 적이 한 번도 없어\n요즘 잘 맞는 거 같던데\n어제 오늘 비...,1:역시 일기예보는 맞은 적이 한 번도 없어\n2:요즘 잘 맞는 거 같던데\n1:어...,일반 대화,일반 대화
1,카카오,요즘엔 군대도 많이 편해 졌다고 하더라\n맞아\n다들 그 얘기 많이 하더라\n모든 ...,1:요즘엔 군대도 많이 편해 졌다고 하더라\n2:맞아\n2:다들 그 얘기 많이 하더...,일반 대화,일반 대화
2,카카오,나 피아노 학원이나 다닐까\n갑자기 뭔 놈에 피아노 학원\n안 친 세월이 몇 년이야...,1:나 피아노 학원이나 다닐까\n2:갑자기 뭔 놈에 피아노 학원\n1:안 친 세월이...,일반 대화,일반 대화
3,카카오,우리 강아지가 자꾸 아무데나 똥을 싸\n교육을 시켜야겠다\n배변 훈련은 어디서 시켜...,1:우리 강아지가 자꾸 아무데나 똥을 싸\n2:교육을 시켜야겠다\n1:배변 훈련은 ...,일반 대화,일반 대화
4,카카오,언니 혼내는 거 아녀요\n원래 혼내면서 배우는 거지\n난 솔직히 아소비 보내고 싶...,1:언니 혼내는 거 아녀요\n2:원래 혼내면서 배우는 거지\n1:난 솔직히 아소비 ...,일반 대화,일반 대화
...,...,...,...,...,...
995,고객센터,네 건의할 게 있어서요\n네 말씀해주십시오 어떤 문의십니까\n인천 송도 달빛공원에 ...,1:안녕하세요 상담원 입니다\n2:네 건의할 게 있어서요\n1:네 말씀해주십시오 어...,일반 대화,일반 대화
996,고객센터,온라인 등급에 대해 궁금합니다\n온라인 등급 어떤 내용이세요\n선정 기간을 알려주세...,1:안녕하세요 상담원 입니다\n2:온라인 등급에 대해 궁금합니다\n1:온라인 등급 ...,일반 대화,일반 대화
997,고객센터,기분 좋은 여행을 위한 여수관광문화 안내 센터\n제가 여수 여행을 준비 중인데요\n...,1:기분 좋은 여행을 위한 여수관광문화 안내 센터 입니다\n2:제가 여수 여행을 준...,일반 대화,일반 대화
998,고객센터,네 수고가 많으세요 혹시 말하기 학습은 어떻게 하나요\n네 저의 어플을 통해 말하기...,1:안녕하세요 상담원 입니다 무엇을 도와드릴까요\n2:네 수고가 많으세요 혹시 말하...,일반 대화,일반 대화


In [35]:
train_1000 = pd.concat([norm_1000.iloc[:,1:], attack_conv])
train_1000

,non_label_sentence,label_sentence,class,binary_class
0,역시 일기예보는 맞은 적이 한 번도 없어\n요즘 잘 맞는 거 같던데\n어제 오늘 비...,1:역시 일기예보는 맞은 적이 한 번도 없어\n2:요즘 잘 맞는 거 같던데\n1:어...,일반 대화,일반 대화
1,요즘엔 군대도 많이 편해 졌다고 하더라\n맞아\n다들 그 얘기 많이 하더라\n모든 ...,1:요즘엔 군대도 많이 편해 졌다고 하더라\n2:맞아\n2:다들 그 얘기 많이 하더...,일반 대화,일반 대화
2,나 피아노 학원이나 다닐까\n갑자기 뭔 놈에 피아노 학원\n안 친 세월이 몇 년이야...,1:나 피아노 학원이나 다닐까\n2:갑자기 뭔 놈에 피아노 학원\n1:안 친 세월이...,일반 대화,일반 대화
3,우리 강아지가 자꾸 아무데나 똥을 싸\n교육을 시켜야겠다\n배변 훈련은 어디서 시켜...,1:우리 강아지가 자꾸 아무데나 똥을 싸\n2:교육을 시켜야겠다\n1:배변 훈련은 ...,일반 대화,일반 대화
4,언니 혼내는 거 아녀요\n원래 혼내면서 배우는 거지\n난 솔직히 아소비 보내고 싶...,1:언니 혼내는 거 아녀요\n2:원래 혼내면서 배우는 거지\n1:난 솔직히 아소비 ...,일반 대화,일반 대화
...,...,...,...,...
3865,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,1:준하야 넌 대가리가 왜이렇게 크냐\n2:내 머리가 뭐\n1:밥먹으면 대가리만 크...,기타 괴롭힘 대화,공격 대화
3866,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,갈취 대화,공격 대화
3867,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,직장 내 괴롭힘 대화,공격 대화
3868,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,1:누구맘대로 여기서 장사하래\n2:이게 무슨일입니까\n1:남의 구역에서 장사하려면...,갈취 대화,공격 대화


In [40]:
train_1000.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_1000_mk2.csv', index=False)

2. 일반 대화 4000개 추가 (카카오 3000 : 고객센터 1000)

In [37]:
kakao_3000 = kakao_data.sample(3000)
kakao_3000

,type,non_label_sentence,label_sentence,class,binary_class
45733,카카오,금연한지 얼마나 됐지\n1년 넘었지\n인간승리다 진짜\n그러게 대신 확실히 입이 ...,1:금연한지 얼마나 됐지\n2:1년 넘었지\n1:인간승리다 진짜\n2:그러게 대신 ...,일반 대화,일반 대화
96079,카카오,너 여자친구 등산 좋아 하냐\n걷는 건 좋아하는데 등산을 좋아하는지는 모르겠네\n너...,1:너 여자친구 등산 좋아 하냐\n2:걷는 건 좋아하는데 등산을 좋아하는지는 모르겠...,일반 대화,일반 대화
60013,카카오,오늘도 에 일이 없어\n널널히 일하면 되겠다\n응 컴퓨터도 고장나서 더 할 일이 없...,1:오늘도 에 일이 없어\n2:널널히 일하면 되겠다\n1:응 컴퓨터도 고장나서 더 ...,일반 대화,일반 대화
96334,카카오,얌 그 소식은 안당 배구 선수\n어 아주 세간의 난리더라\n걔 자기 언니한테 욕한거...,1:얌 그 소식은 안당 배구 선수\n2:어 아주 세간의 난리더라\n1:걔 자기 언니...,일반 대화,일반 대화
16320,카카오,어제 날씨 좀 따뜻하던데 하늘이 많이 어둡더라\n어제 안개 낀 거 처럼 뿌옇더라\n...,1:어제 날씨 좀 따뜻하던데 하늘이 많이 어둡더라\n2:어제 안개 낀 거 처럼 뿌옇...,일반 대화,일반 대화
...,...,...,...,...,...
17127,카카오,저는 이제 영화 타임에요\n오늘의 영화는 무엇인고\n넷플렉스에 있는 낙원의 밤이요\...,1:저는 이제 영화 타임에요\n2:오늘의 영화는 무엇인고\n1:넷플렉스에 있는 낙원...,일반 대화,일반 대화
59097,카카오,요즘에 코로나 시국이라 군대 제대할 때 미복귀 전역하더라\n그렇게 시행된지 꽤 됐을...,1:요즘에 코로나 시국이라 군대 제대할 때 미복귀 전역하더라\n2:그렇게 시행된지 ...,일반 대화,일반 대화
15619,카카오,여행 계획 있음\n흠 12월 즈음에\n난 클마에 어디 가지\n생각 중\n육지를...,1:여행 계획 있음\n2:흠 12월 즈음에\n1:난 클마에 어디 가지\n2:생각 중...,일반 대화,일반 대화
93365,카카오,요새 양세찬 웃기지 않냐\n나는 김수로가 좋더라고\n재밌고\n김수로\n요새 티비 안...,1:요새 양세찬 웃기지 않냐\n2:나는 김수로가 좋더라고\n2:재밌고\n1:김수로\...,일반 대화,일반 대화


In [38]:
cs_1000 = cs_data.sample(1000)
cs_1000

,type,non_label_sentence,label_sentence,class,binary_class
5701,고객센터,네 혹시 소리산 코스 이용하려고 하는데요\n네 당일바로 이용이 가능하구요 양평 산...,1: 입니다 무엇을 도와드릴까요\n2:네 안녕하세요 혹시 소리산 코스 이용하려고 하...,일반 대화,일반 대화
67869,고객센터,네 하나 물어볼게요\n주택 임대차 계약신고하는데 서류 뭐 준비해야 해요\n주택 임...,1:안녕하세요 입니다\n2:네 안녕하세요 하나 물어볼게요\n2:주택 임대차 계약신고...,일반 대화,일반 대화
78253,고객센터,제가 배송을 언제쯤 받을 수 있을지 궁금해서요\n페이지에 배송지 입력하실 때 예상수...,1:안녕하세요 무엇을 도와드릴까요\n2:제가 배송을 언제쯤 받을 수 있을지 궁금해서...,일반 대화,일반 대화
83620,고객센터,궁금한 게 있어서 연락드립니다\n네\n에서 광고가 너무 많이 오네요\n혹시 광고성 ...,1:안녕하세요 상담원 입니다\n2:안녕하세요 궁금한 게 있어서 연락드립니다\n1:네...,일반 대화,일반 대화
3792,고객센터,역사가 있는\n네 저 골프장 위치 문의 좀 드리고 싶어서요\n네 어떤 골프장 안내...,1:반갑습니다 역사가 있는 상담원 입니다\n2:네 저 골프장 위치 문의 좀 드리고 ...,일반 대화,일반 대화
...,...,...,...,...,...
65199,고객센터,물어볼 게 있어서 연락했어요\n신속하고 정확하게 답변해 드리겠습니다\n항만 내에서 ...,1:반갑습니다 상담원 입니다\n2:안녕하세요 물어볼 게 있어서 연락했어요\n1:신속...,일반 대화,일반 대화
3739,고객센터,문의 주셔서\n저희 남원 음식점 추천해 주세요\n네 어떤 음식 추천을\n혹시 야외에...,1:문의 주셔서 감사합니다 입니다\n2:저희 남원 음식점 추천해 주세요\n1:네 어...,일반 대화,일반 대화
76130,고객센터,고객 센터\n어떤 도움 필요하세요\n결제를 했거든요 제가\n결제 방식을 바꾸고 싶은...,1:안녕하세요 고객 센터입니다\n1:어떤 도움 필요하세요\n2: 결제를 했거든요 제...,일반 대화,일반 대화
89312,고객센터,네 제가 학습공간 예약을 했는데 변경할 수 있나 해서요\n네 이용 시작 시간 전이신...,1: 안녕하세요 상담원 입니다\n2: 네 제가 학습공간 예약을 했는데 변경할 수 있...,일반 대화,일반 대화


In [39]:
norm_4000 = pd.concat([kakao_3000, cs_1000], ignore_index=True)
norm_4000

,type,non_label_sentence,label_sentence,class,binary_class
0,카카오,금연한지 얼마나 됐지\n1년 넘었지\n인간승리다 진짜\n그러게 대신 확실히 입이 ...,1:금연한지 얼마나 됐지\n2:1년 넘었지\n1:인간승리다 진짜\n2:그러게 대신 ...,일반 대화,일반 대화
1,카카오,너 여자친구 등산 좋아 하냐\n걷는 건 좋아하는데 등산을 좋아하는지는 모르겠네\n너...,1:너 여자친구 등산 좋아 하냐\n2:걷는 건 좋아하는데 등산을 좋아하는지는 모르겠...,일반 대화,일반 대화
2,카카오,오늘도 에 일이 없어\n널널히 일하면 되겠다\n응 컴퓨터도 고장나서 더 할 일이 없...,1:오늘도 에 일이 없어\n2:널널히 일하면 되겠다\n1:응 컴퓨터도 고장나서 더 ...,일반 대화,일반 대화
3,카카오,얌 그 소식은 안당 배구 선수\n어 아주 세간의 난리더라\n걔 자기 언니한테 욕한거...,1:얌 그 소식은 안당 배구 선수\n2:어 아주 세간의 난리더라\n1:걔 자기 언니...,일반 대화,일반 대화
4,카카오,어제 날씨 좀 따뜻하던데 하늘이 많이 어둡더라\n어제 안개 낀 거 처럼 뿌옇더라\n...,1:어제 날씨 좀 따뜻하던데 하늘이 많이 어둡더라\n2:어제 안개 낀 거 처럼 뿌옇...,일반 대화,일반 대화
...,...,...,...,...,...
3995,고객센터,물어볼 게 있어서 연락했어요\n신속하고 정확하게 답변해 드리겠습니다\n항만 내에서 ...,1:반갑습니다 상담원 입니다\n2:안녕하세요 물어볼 게 있어서 연락했어요\n1:신속...,일반 대화,일반 대화
3996,고객센터,문의 주셔서\n저희 남원 음식점 추천해 주세요\n네 어떤 음식 추천을\n혹시 야외에...,1:문의 주셔서 감사합니다 입니다\n2:저희 남원 음식점 추천해 주세요\n1:네 어...,일반 대화,일반 대화
3997,고객센터,고객 센터\n어떤 도움 필요하세요\n결제를 했거든요 제가\n결제 방식을 바꾸고 싶은...,1:안녕하세요 고객 센터입니다\n1:어떤 도움 필요하세요\n2: 결제를 했거든요 제...,일반 대화,일반 대화
3998,고객센터,네 제가 학습공간 예약을 했는데 변경할 수 있나 해서요\n네 이용 시작 시간 전이신...,1: 안녕하세요 상담원 입니다\n2: 네 제가 학습공간 예약을 했는데 변경할 수 있...,일반 대화,일반 대화


In [41]:
train_4000 = pd.concat([norm_4000.iloc[:,1:], attack_conv])
train_4000

,non_label_sentence,label_sentence,class,binary_class
0,금연한지 얼마나 됐지\n1년 넘었지\n인간승리다 진짜\n그러게 대신 확실히 입이 ...,1:금연한지 얼마나 됐지\n2:1년 넘었지\n1:인간승리다 진짜\n2:그러게 대신 ...,일반 대화,일반 대화
1,너 여자친구 등산 좋아 하냐\n걷는 건 좋아하는데 등산을 좋아하는지는 모르겠네\n너...,1:너 여자친구 등산 좋아 하냐\n2:걷는 건 좋아하는데 등산을 좋아하는지는 모르겠...,일반 대화,일반 대화
2,오늘도 에 일이 없어\n널널히 일하면 되겠다\n응 컴퓨터도 고장나서 더 할 일이 없...,1:오늘도 에 일이 없어\n2:널널히 일하면 되겠다\n1:응 컴퓨터도 고장나서 더 ...,일반 대화,일반 대화
3,얌 그 소식은 안당 배구 선수\n어 아주 세간의 난리더라\n걔 자기 언니한테 욕한거...,1:얌 그 소식은 안당 배구 선수\n2:어 아주 세간의 난리더라\n1:걔 자기 언니...,일반 대화,일반 대화
4,어제 날씨 좀 따뜻하던데 하늘이 많이 어둡더라\n어제 안개 낀 거 처럼 뿌옇더라\n...,1:어제 날씨 좀 따뜻하던데 하늘이 많이 어둡더라\n2:어제 안개 낀 거 처럼 뿌옇...,일반 대화,일반 대화
...,...,...,...,...
3865,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,1:준하야 넌 대가리가 왜이렇게 크냐\n2:내 머리가 뭐\n1:밥먹으면 대가리만 크...,기타 괴롭힘 대화,공격 대화
3866,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,갈취 대화,공격 대화
3867,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,직장 내 괴롭힘 대화,공격 대화
3868,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,1:누구맘대로 여기서 장사하래\n2:이게 무슨일입니까\n1:남의 구역에서 장사하려면...,갈취 대화,공격 대화


In [42]:
train_4000.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_4000.csv', index=False)

3. 일반 대화 10000개 추가 (카카오 7500 : 고객센터 2500)

In [43]:
kakao_7500 = kakao_data.sample(7500)
kakao_7500

,type,non_label_sentence,label_sentence,class,binary_class
104560,카카오,해병대는 자기가 지원해서 가는 거잖아\n응응 시험 같은 것도 있지 않아\n어떻게 해...,1:해병대는 자기가 지원해서 가는 거잖아\n2:응응 시험 같은 것도 있지 않아\n1...,일반 대화,일반 대화
103523,카카오,오늘 서울은 하루종일 꾸물하다 날씨가\n요즘 날씨가 계속 꾸물하고 비오고\n언제 선...,1:오늘 서울은 하루종일 꾸물하다 날씨가\n2:요즘 날씨가 계속 꾸물하고 비오고\n...,일반 대화,일반 대화
100554,카카오,남자들은 전립선 비대증이 큰일이에요\n진작에 관리 안 한 걸 후회한다\n치료도 잘 ...,1:남자들은 전립선 비대증이 큰일이에요\n2:진작에 관리 안 한 걸 후회한다\n1:...,일반 대화,일반 대화
15917,카카오,안녕\n나는 이 날씨에 서핑하러 간다\n와 이제 곧 겨울인데 서핑을 해\n당연히 제...,1:안녕\n1:나는 이 날씨에 서핑하러 간다\n2:와 이제 곧 겨울인데 서핑을 해\...,일반 대화,일반 대화
47007,카카오,넌 몇살에 결혼 하고 싶어\n35살 난 최대한 즐기고 결혼 할 거야\n그럼 애기는 ...,1:넌 몇살에 결혼 하고 싶어\n2:35살 난 최대한 즐기고 결혼 할 거야\n1:그...,일반 대화,일반 대화
...,...,...,...,...,...
14954,카카오,하이 우리 사촌 누나 이번에 결혼하는데 축의금 얼마 해 줘야 하냐\n오랜만이다\n...,1:하이 우리 사촌 누나 이번에 결혼하는데 축의금 얼마 해 줘야 하냐\n2:오랜만이...,일반 대화,일반 대화
103042,카카오,오빠야 고양이 알러지는 없음않\n알러지는 딱히 없는 것 같다\n오 다행이네\n고양이...,1:오빠야 고양이 알러지는 없음않\n2:알러지는 딱히 없는 것 같다\n1: 오 다행...,일반 대화,일반 대화
34465,카카오,유전자의 신비란\n나 우리엄마 어릴때 사진 봤는데 내 어릴때 사진이랑 똑같이 생겼어...,1:유전자의 신비란\n1:나 우리엄마 어릴때 사진 봤는데 내 어릴때 사진이랑 똑같이...,일반 대화,일반 대화
16637,카카오,손흥민이 메시를 넘었다네\n어어 헐헐 진짜가\n몸값이 메시 다들 알지\n그 정...,1:손흥민이 메시를 넘었다네\n2: 어어 헐헐 진짜가\n1:몸값이 메시 다들 알지\...,일반 대화,일반 대화


In [44]:
cs_2500 = cs_data.sample(2500)
cs_2500

,type,non_label_sentence,label_sentence,class,binary_class
73751,고객센터,고객센터\n에어컨 이전 설치 하고 싶어서 전화 했는데요\n이사로 인한 이전 설치 맞...,1:반갑습니다 고객센터 입니다\n2:에어컨 이전 설치 하고 싶어서 전화 했는데요\n...,일반 대화,일반 대화
72428,고객센터,티비 고장 건으로 불편드려 너무 죄송하고요\n어떻게 삼 년밖에 안되서 이렇게 심하게...,1: 티비 고장 건으로 불편드려 너무 죄송하고요\n2:어떻게 삼 년밖에 안되서 이렇...,일반 대화,일반 대화
637,고객센터,상쾌한 하루시작\n혹시 애견동반 가능하고 픽업 서비스도 되는 펜션이 있을까요\n케렌...,1:상쾌한 하루시작 상담원 입니다\n2:혹시 애견동반 가능하고 픽업 서비스도 되는 ...,일반 대화,일반 대화
66623,고객센터,안녕하십니까 친절한\n예 수고가 많으십니다 제가 질문사항이 있어서 전화드렸어요\n네...,1:안녕하십니까 친절한 상담원 입니다\n2:예 수고가 많으십니다 제가 질문사항이 있...,일반 대화,일반 대화
91214,고객센터,고객센터\n네 문의 사항이 있어서 전화 드렸어요\n네\n제가 문의글을 남겼는데 이...,1: 반갑습니다 고객센터 입니다\n2: 네 안녕하세요 문의 사항이 있어서 전화 드렸...,일반 대화,일반 대화
...,...,...,...,...,...
90891,고객센터,도와 드릴까요\n제가 지난주에 여기서 평생교육원에 예약을 했는데요\n네 학습 공간을...,1:안녕하세요 상담원 입니다 무엇을 도와 드릴까요\n2:제가 지난주에 여기서 평생교...,일반 대화,일반 대화
89863,고객센터,무엇이든 도와드리겠습니다\n저 학습기기 구매했는데요\n학습기기 구매 시 보너스 캐시...,1:무엇이든 도와드리겠습니다 상담원 입니다\n2:저 학습기기 구매했는데요\n2:학습...,일반 대화,일반 대화
83396,고객센터,네 여쭤볼게 있어서 연락드렸어요\n네 말씀 주시면 안내드리겠습니다\n쇼핑라이브 적...,1:안녕하세요 입니다\n2:네 안녕하세요 여쭤볼게 있어서 연락드렸어요\n1:네 말씀...,일반 대화,일반 대화
75744,고객센터,고객센터\n의 질문에 성심껏 답변해 드리겠습니다\n네 에스형과 디피형이 무슨 차이가...,1:안녕하세요 고객센터입니다\n1:의 질문에 성심껏 답변해 드리겠습니다\n2:네 에...,일반 대화,일반 대화


In [45]:
norm_10000 = pd.concat([kakao_7500, cs_2500], ignore_index=True)
norm_10000

,type,non_label_sentence,label_sentence,class,binary_class
0,카카오,해병대는 자기가 지원해서 가는 거잖아\n응응 시험 같은 것도 있지 않아\n어떻게 해...,1:해병대는 자기가 지원해서 가는 거잖아\n2:응응 시험 같은 것도 있지 않아\n1...,일반 대화,일반 대화
1,카카오,오늘 서울은 하루종일 꾸물하다 날씨가\n요즘 날씨가 계속 꾸물하고 비오고\n언제 선...,1:오늘 서울은 하루종일 꾸물하다 날씨가\n2:요즘 날씨가 계속 꾸물하고 비오고\n...,일반 대화,일반 대화
2,카카오,남자들은 전립선 비대증이 큰일이에요\n진작에 관리 안 한 걸 후회한다\n치료도 잘 ...,1:남자들은 전립선 비대증이 큰일이에요\n2:진작에 관리 안 한 걸 후회한다\n1:...,일반 대화,일반 대화
3,카카오,안녕\n나는 이 날씨에 서핑하러 간다\n와 이제 곧 겨울인데 서핑을 해\n당연히 제...,1:안녕\n1:나는 이 날씨에 서핑하러 간다\n2:와 이제 곧 겨울인데 서핑을 해\...,일반 대화,일반 대화
4,카카오,넌 몇살에 결혼 하고 싶어\n35살 난 최대한 즐기고 결혼 할 거야\n그럼 애기는 ...,1:넌 몇살에 결혼 하고 싶어\n2:35살 난 최대한 즐기고 결혼 할 거야\n1:그...,일반 대화,일반 대화
...,...,...,...,...,...
9995,고객센터,도와 드릴까요\n제가 지난주에 여기서 평생교육원에 예약을 했는데요\n네 학습 공간을...,1:안녕하세요 상담원 입니다 무엇을 도와 드릴까요\n2:제가 지난주에 여기서 평생교...,일반 대화,일반 대화
9996,고객센터,무엇이든 도와드리겠습니다\n저 학습기기 구매했는데요\n학습기기 구매 시 보너스 캐시...,1:무엇이든 도와드리겠습니다 상담원 입니다\n2:저 학습기기 구매했는데요\n2:학습...,일반 대화,일반 대화
9997,고객센터,네 여쭤볼게 있어서 연락드렸어요\n네 말씀 주시면 안내드리겠습니다\n쇼핑라이브 적...,1:안녕하세요 입니다\n2:네 안녕하세요 여쭤볼게 있어서 연락드렸어요\n1:네 말씀...,일반 대화,일반 대화
9998,고객센터,고객센터\n의 질문에 성심껏 답변해 드리겠습니다\n네 에스형과 디피형이 무슨 차이가...,1:안녕하세요 고객센터입니다\n1:의 질문에 성심껏 답변해 드리겠습니다\n2:네 에...,일반 대화,일반 대화


In [46]:
train_10000 = pd.concat([norm_10000.iloc[:, 1:], attack_conv], ignore_index=True)
train_10000

,non_label_sentence,label_sentence,class,binary_class
0,해병대는 자기가 지원해서 가는 거잖아\n응응 시험 같은 것도 있지 않아\n어떻게 해...,1:해병대는 자기가 지원해서 가는 거잖아\n2:응응 시험 같은 것도 있지 않아\n1...,일반 대화,일반 대화
1,오늘 서울은 하루종일 꾸물하다 날씨가\n요즘 날씨가 계속 꾸물하고 비오고\n언제 선...,1:오늘 서울은 하루종일 꾸물하다 날씨가\n2:요즘 날씨가 계속 꾸물하고 비오고\n...,일반 대화,일반 대화
2,남자들은 전립선 비대증이 큰일이에요\n진작에 관리 안 한 걸 후회한다\n치료도 잘 ...,1:남자들은 전립선 비대증이 큰일이에요\n2:진작에 관리 안 한 걸 후회한다\n1:...,일반 대화,일반 대화
3,안녕\n나는 이 날씨에 서핑하러 간다\n와 이제 곧 겨울인데 서핑을 해\n당연히 제...,1:안녕\n1:나는 이 날씨에 서핑하러 간다\n2:와 이제 곧 겨울인데 서핑을 해\...,일반 대화,일반 대화
4,넌 몇살에 결혼 하고 싶어\n35살 난 최대한 즐기고 결혼 할 거야\n그럼 애기는 ...,1:넌 몇살에 결혼 하고 싶어\n2:35살 난 최대한 즐기고 결혼 할 거야\n1:그...,일반 대화,일반 대화
...,...,...,...,...
13865,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,1:준하야 넌 대가리가 왜이렇게 크냐\n2:내 머리가 뭐\n1:밥먹으면 대가리만 크...,기타 괴롭힘 대화,공격 대화
13866,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,갈취 대화,공격 대화
13867,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,직장 내 괴롭힘 대화,공격 대화
13868,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,1:누구맘대로 여기서 장사하래\n2:이게 무슨일입니까\n1:남의 구역에서 장사하려면...,갈취 대화,공격 대화


In [47]:
train_10000.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_10000_mk2.csv', index=False)